M5 Forecasting - Accuracy

Note: This is one of the two complementary competitions that together comprise the M5 forecasting challenge. Can you estimate, as precisely as possible, the point forecasts of the unit sales of various products sold in the USA by Walmart? If you are interested in estimating the uncertainty distribution of the realized values of the same series, be sure to check out its companion competition

How much camping gear will one store sell each month in a year? To the uninitiated, calculating sales at this level may seem as difficult as predicting the weather. Both types of forecasting rely on science and historical data. While a wrong weather forecast may result in you carrying around an umbrella on a sunny day, inaccurate business forecasts could result in actual or opportunity losses. In this competition, in addition to traditional forecasting methods you’re also challenged to use machine learning to improve forecast accuracy.

The Makridakis Open Forecasting Center (MOFC) at the University of Nicosia conducts cutting-edge forecasting research and provides business forecast training. It helps companies achieve accurate predictions, estimate the levels of uncertainty, avoiding costly mistakes, and apply best forecasting practices. The MOFC is well known for its Makridakis Competitions, the first of which ran in the 1980s.

In this competition, the fifth iteration, you will use hierarchical sales data from Walmart, the world’s largest company by revenue, to forecast daily sales for the next 28 days. The data, covers stores in three US States (California, Texas, and Wisconsin) and includes item level, department, product categories, and store details. In addition, it has explanatory variables such as price, promotions, day of the week, and special events. Together, this robust dataset can be used to improve forecasting accuracy.

If successful, your work will continue to advance the theory and practice of forecasting. The methods used can be applied in various business areas, such as setting up appropriate inventory or service levels. Through its business support and training, the MOFC will help distribute the tools and knowledge so others can achieve more accurate and better calibrated forecasts, reduce waste and be able to appreciate uncertainty and its risk implications.

Evaluation:
This competition uses a Weighted Root Mean Squared Scaled Error (RMSSE). Extensive details about the metric, scaling, and weighting can be found in the [M5 Participants Guide](https://mofc.unic.ac.cy/m5-competition/).

In [1]:
import gc
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from statsmodels.tsa.stattools import adfuller

pd.options.display.max_columns = 30

In [2]:
'''
Function to reduce memory usage.
From: https://www.kaggle.com/ragnar123/very-fst-model
'''
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
#Load data.
calendar = pd.read_csv('calendar.csv')
calendar = reduce_mem_usage(calendar)

#verify how many weeks do we have. This will help us to analise the prices.
print('weeks: ', len(calendar['wm_yr_wk'].drop_duplicates()))
print()

#Verify null
print('Verify null:')
print(calendar.isnull().sum())
print()
print(calendar.isnull().sum() / len(calendar))
print()
calendar.head()

Mem. usage decreased to  0.12 Mb (41.9% reduction)
weeks:  282

Verify null:
date               0
wm_yr_wk           0
weekday            0
wday               0
month              0
year               0
d                  0
event_name_1    1807
event_type_1    1807
event_name_2    1964
event_type_2    1964
snap_CA            0
snap_TX            0
snap_WI            0
dtype: int64

date            0.000000
wm_yr_wk        0.000000
weekday         0.000000
wday            0.000000
month           0.000000
year            0.000000
d               0.000000
event_name_1    0.917725
event_type_1    0.917725
event_name_2    0.997461
event_type_2    0.997461
snap_CA         0.000000
snap_TX         0.000000
snap_WI         0.000000
dtype: float64



,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


File 1: “calendar.csv”

Contains information about the dates the products are sold.

     date: The date in a “y-m-d” format.

     wm_yr_wk: The id of the week the date belongs to.
    
     weekday: The type of the day (Saturday, Sunday, …, Friday).
    
     wday: The id of the weekday, starting from Saturday.
    
     month: The month of the date.
    
     year: The year of the date.
    
     event_name_1: If the date includes an event, the name of this event.
    
     event_type_1: If the date includes an event, the type of this event.
    
     event_name_2: If the date includes a second event, the name of this event.
    
     event_type_2: If the date includes a second event, the type of this event.
    
     snap_CA, snap_TX, and snap_WI: A binary variable (0 or 1) indicating whether the stores of CA, TX or WI allow SNAP2 purchases on the examined date. 1 indicates that SNAP purchases are allowed.

In [4]:
sales_train_validation = pd.read_csv('sales_train_validation.csv')
sales_train_validation = reduce_mem_usage(sales_train_validation)

#Verify the quantity of items and store. This will help us to see how many weeks we do not have any prices for a product.
print('Items and stores:', len(sales_train_validation[['item_id', 'store_id']].drop_duplicates()))
print()

#Verify null
print('Verify null:')
print(sales_train_validation.isnull().sum()[sales_train_validation.isnull().sum() > 0]) #there is not any null values.
print()
print(sales_train_validation.isnull().sum()[sales_train_validation.isnull().sum() > 0] / len(sales_train_validation))
print()
sales_train_validation.head()

Mem. usage decreased to 95.00 Mb (78.7% reduction)
Items and stores: 30490

Verify null:
Series([], dtype: int64)

Series([], dtype: float64)



,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,...,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,...,2,0,0,0,1,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,...,0,0,0,1,2,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,...,4,2,0,3,0,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,...,1,2,0,1,1,2,1,1,0,1,1,2,2,2,4


File 3: “sales_train.csv”

Contains the historical daily unit sales data per product and store.

     item_id: The id of the product.

     dept_id: The id of the department the product belongs to.

     cat_id: The id of the category the product belongs to.

     store_id: The id of the store where the product is sold.

     state_id: The State where the store is located.

     d_1, d_2, …, d_i, … d_1941: The number of units sold at day i, starting from 2011-01-29.
    
    
We have 282 and 30,490 differents items. Therefore, the ideal would be to have 8,598,180 (282 x 30,400) prices. 

In [5]:
sell_prices = pd.read_csv('sell_prices.csv')
sell_prices = reduce_mem_usage(sell_prices)

print("len: ", len(sell_prices))
print()
#Verify null
print('Verify null:')
print(sell_prices.isnull().sum())
print()
print(sell_prices.isnull().sum() / len(sell_prices))
print()
display(sell_prices.head())
print()

Mem. usage decreased to 130.48 Mb (37.5% reduction)
len:  6841121

Verify null:
store_id      0
item_id       0
wm_yr_wk      0
sell_price    0
dtype: int64

store_id      0.0
item_id       0.0
wm_yr_wk      0.0
sell_price    0.0
dtype: float64



,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.578125
1,CA_1,HOBBIES_1_001,11326,9.578125
2,CA_1,HOBBIES_1_001,11327,8.257812
3,CA_1,HOBBIES_1_001,11328,8.257812
4,CA_1,HOBBIES_1_001,11329,8.257812


File 2: “sell_prices.csv”

Contains information about the price of the products sold per store and date.

     store_id: The id of the store where the product is sold.

     item_id: The id of the product.

     wm_yr_wk: The id of the week.

     sell_price: The price of the product for the given week/store. The price is provided per week (average across seven days). If not available, this means that the product was not sold during the examined week. Note that although prices are constant at weekly basis, they may change through time (both training and test set).
    
    
Considering that we have only 6,841,121 prices, there are  1,757,059 (8,598,180 - 6,841,121) products that were not solds in a given week.

In [6]:
#Define 2 holdout
holdout1 = pd.read_csv('sample_submission.csv')
holdout1 = reduce_mem_usage(holdout1)
holdout2 = pd.read_csv('sample_submission.csv')
holdout2 = reduce_mem_usage(holdout2)


#One holdout is related to forecast between day 1914 and 1941.
#The second, between day 1942 and 1969
test1 = holdout1.copy()
test2 = holdout2.copy()
test1.columns = ['id', 'd_1914', 'd_1915', 'd_1916', 'd_1917', 'd_1918', 'd_1919', 'd_1920', 'd_1921', 'd_1922', 'd_1923',
                    'd_1924', 'd_1925', 'd_1926', 'd_1927', 'd_1928', 'd_1929', 'd_1930', 'd_1931', 
                    'd_1932', 'd_1933', 'd_1934', 'd_1935', 'd_1936', 'd_1937', 'd_1938', 'd_1939', 'd_1940', 'd_1941']
test2.columns = ['id', 'd_1942', 'd_1943', 'd_1944', 'd_1945', 'd_1946', 'd_1947', 'd_1948', 'd_1949', 'd_1950', 'd_1951', 
                    'd_1952', 'd_1953', 'd_1954', 'd_1955', 'd_1956', 'd_1957', 'd_1958', 'd_1959', 
                    'd_1960', 'd_1961', 'd_1962', 'd_1963', 'd_1964', 'd_1965', 'd_1966', 'd_1967', 'd_1968', 'd_1969']

print('Shape test1: ' + str(test1.shape))  #Observation: each table has all items with "_validation", and after it repeats the items with "_evaluation"
print('Shape test2: ' + str(test2.shape))
display(test1.head())
display(test2.head())

Mem. usage decreased to  2.09 Mb (84.5% reduction)
Mem. usage decreased to  2.09 Mb (84.5% reduction)
Shape test1: (60980, 29)
Shape test2: (60980, 29)


,id,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,d_1929,d_1930,d_1931,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


,id,d_1942,d_1943,d_1944,d_1945,d_1946,d_1947,d_1948,d_1949,d_1950,d_1951,d_1952,d_1953,d_1954,d_1955,d_1956,d_1957,d_1958,d_1959,d_1960,d_1961,d_1962,d_1963,d_1964,d_1965,d_1966,d_1967,d_1968,d_1969
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
#Transpose sales_train_validation so the days goes to rows.
sales_train_validation = pd.melt(sales_train_validation, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')

#Verify null
print(sales_train_validation.isnull().sum())
print()
print(sales_train_validation.isnull().sum() / len(sales_train_validation))
print()
display(sales_train_validation.head())
print('...')
display(sales_train_validation.tail())

id          0
item_id     0
dept_id     0
cat_id      0
store_id    0
state_id    0
day         0
demand      0
dtype: int64

id          0.0
item_id     0.0
dept_id     0.0
cat_id      0.0
store_id    0.0
state_id    0.0
day         0.0
demand      0.0
dtype: float64



,id,item_id,dept_id,cat_id,store_id,state_id,day,demand
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0


...


,id,item_id,dept_id,cat_id,store_id,state_id,day,demand
58327365,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1913,1
58327366,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1913,0
58327367,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1913,0
58327368,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1913,3
58327369,FOODS_3_827_WI_3_validation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,d_1913,0


In [8]:
#Concat our train database with the holdouts.
#Considering that we do not have any information for days after 1913, this wil not imply in any kind of information leak. 

#Get information about the product id.
product_id = sales_train_validation[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']].drop_duplicates()

# merge with product table
validation_bool = test1['id'].str.contains("_validation") 
test1 = test1[validation_bool]
test1 = test1.merge(product_id, how = 'left', on = 'id')
test2 = test2[validation_bool]
test2 = test2.merge(product_id, how = 'left', on = 'id')

#Transpose so the days goes to rows.
test1 = pd.melt(test1, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
test2 = pd.melt(test2, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')

#Define what is train and what is test.
sales_train_validation['part'] = 'train'
test1['part'] = 'test1'
test2['part'] = 'test2'

In [9]:
#Concat
data = pd.concat([sales_train_validation, test1, test2], axis = 0)
#del sales_train_validation, test1, test2

#Verify null
print(data.isnull().sum())
print()
print(data.isnull().sum() / len(data))
print()
data.info()

id          0
item_id     0
dept_id     0
cat_id      0
store_id    0
state_id    0
day         0
demand      0
part        0
dtype: int64

id          0.0
item_id     0.0
dept_id     0.0
cat_id      0.0
store_id    0.0
state_id    0.0
day         0.0
demand      0.0
part        0.0
dtype: float64

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60034810 entries, 0 to 853719
Data columns (total 9 columns):
id          object
item_id     object
dept_id     object
cat_id      object
store_id    object
state_id    object
day         object
demand      int16
part        object
dtypes: int16(1), object(8)
memory usage: 4.1+ GB


About the products:
The M5 dataset, generously made available by Walmart, involves the unit sales of various products sold in the USA, organized in the form of grouped time series. More specifically, the dataset involves the unit sales of 3,075 products, classified in 3 product categories (Hobbies, Foods, and Household) and 7 product departments, in which the above-mentioned categories are disaggregated. The products are sold across 10 stores, located in 3 States (CA, TX, and WI). In this respect, the bottom-level of the hierarchy, i.e., product-store unit sales, can be mapped either across product categories or geographical regions. More information can be found in the [M5 Participants Guide](https://mofc.unic.ac.cy/m5-competition/).

In [10]:
#Verify the numbers described above.

products = product_id['item_id'].value_counts(dropna = False) #3075 There is only 3049. Howeveer it is possible that M5's test database has these missing products.
departments = product_id['dept_id'].value_counts(dropna = False) #7 OK!
categories = product_id['cat_id'].value_counts(dropna = False) #3 OK!
stores = product_id['store_id'].value_counts(dropna = False) #10 OK!
States = product_id['state_id'].value_counts(dropna = False) #3 OK!

print('States:')
print('')
display(States)
print('')
print('stores:')
print('')
display(stores)
print('')
print('categories:')
print('')
display(categories)
print('')
print('departments:')
print('')
display(departments)
print('')
print('products:')
print('')
display(products)

States:



CA    12196
TX     9147
WI     9147
Name: state_id, dtype: int64


stores:



CA_2    3049
CA_1    3049
CA_3    3049
TX_1    3049
TX_3    3049
TX_2    3049
WI_1    3049
WI_3    3049
CA_4    3049
WI_2    3049
Name: store_id, dtype: int64


categories:



FOODS        14370
HOUSEHOLD    10470
HOBBIES       5650
Name: cat_id, dtype: int64


departments:



FOODS_3        8230
HOUSEHOLD_1    5320
HOUSEHOLD_2    5150
HOBBIES_1      4160
FOODS_2        3980
FOODS_1        2160
HOBBIES_2      1490
Name: dept_id, dtype: int64


products:



HOBBIES_1_016      10
FOODS_3_485        10
FOODS_2_129        10
HOBBIES_2_118      10
FOODS_3_312        10
                   ..
HOUSEHOLD_2_376    10
HOUSEHOLD_1_513    10
HOBBIES_1_247      10
HOUSEHOLD_1_399    10
HOUSEHOLD_2_464    10
Name: item_id, Length: 3049, dtype: int64

In [11]:
#Verify holidays
event_name_1 = calendar['event_name_1'].value_counts(dropna = False) #30
event_type_1 = calendar['event_type_1'].value_counts(dropna = False) #4
event_name_2 = calendar['event_name_2'].value_counts(dropna = False) #4
event_type_2 = calendar['event_type_2'].value_counts(dropna = False) #2


#It shoud be binary variables:
snap_CA = calendar['snap_CA'].value_counts(dropna = False) #49.28% are snap days
snap_TX = calendar['snap_TX'].value_counts(dropna = False)
snap_WI = calendar['snap_WI'].value_counts(dropna = False)

print('event_name_1:')
print('')
display(event_name_1)
print('')
print('event_type_1:')
print('')
display(event_type_1)
print('')
print('event_name_2:')
print('')
display(event_name_2)
print('')
print('event_type_2:')
print('')
display(event_type_2)
print('')
print('snap_CA:')
print('')
display(snap_CA)
print('')
print('snap_TX:')
print('')
display(snap_TX)
print('')
print('snap_WI:')
print('')
display(snap_WI)

event_name_1:



NaN                    1807
LentWeek2                 6
Pesach End                6
LentStart                 6
NBAFinalsStart            6
Ramadan starts            6
ValentinesDay             6
PresidentsDay             6
MemorialDay               6
Mother's day              6
NBAFinalsEnd              6
Purim End                 6
StPatricksDay             6
SuperBowl                 6
Chanukah End              5
OrthodoxChristmas         5
Christmas                 5
Halloween                 5
VeteransDay               5
ColumbusDay               5
Easter                    5
IndependenceDay           5
Eid al-Fitr               5
LaborDay                  5
NewYear                   5
MartinLutherKingDay       5
Cinco De Mayo             5
OrthodoxEaster            5
EidAlAdha                 5
Thanksgiving              5
Father's day              4
Name: event_name_1, dtype: int64


event_type_1:



NaN          1807
Religious      55
National       52
Cultural       37
Sporting       18
Name: event_type_1, dtype: int64


event_name_2:



...

0    1319
1     650
Name: snap_CA, dtype: int64


snap_TX:



0    1319
1     650
Name: snap_TX, dtype: int64


snap_WI:



0    1319
1     650
Name: snap_WI, dtype: int64

In [12]:
#Add information in our dataset.
#Merge with calendar.
data = pd.merge(data, calendar, how = 'left', left_on = ['day'], right_on = ['d'])
data.drop(['d'], inplace = True, axis = 1)

#Verify missing.
print(data.isnull().sum())
print()
print(data.isnull().sum() / len(data))

id                     0
item_id                0
dept_id                0
cat_id                 0
store_id               0
state_id               0
day                    0
demand                 0
part                   0
date                   0
wm_yr_wk               0
weekday                0
wday                   0
month                  0
year                   0
event_name_1    55095430
event_type_1    55095430
event_name_2    59882360
event_type_2    59882360
snap_CA                0
snap_TX                0
snap_WI                0
dtype: int64

id              0.000000
item_id         0.000000
dept_id         0.000000
cat_id          0.000000
store_id        0.000000
state_id        0.000000
day             0.000000
demand          0.000000
part            0.000000
date            0.000000
wm_yr_wk        0.000000
weekday         0.000000
wday            0.000000
month           0.000000
year            0.000000
event_name_1    0.917725
event_type_1    0.917725
event_name_

In [13]:
#Merge with sell_prices.
data = pd.merge(data, sell_prices, how = 'left', on = ['store_id', 'item_id', 'wm_yr_wk'])

#Verify missing.
print(data.isnull().sum())
print()
print(data.isnull().sum() / len(data))

print()
print(data.info())
display(data.head())

id                     0
item_id                0
dept_id                0
cat_id                 0
store_id               0
state_id               0
day                    0
demand                 0
part                   0
date                   0
wm_yr_wk               0
weekday                0
wday                   0
month                  0
year                   0
event_name_1    55095430
event_type_1    55095430
event_name_2    59882360
event_type_2    59882360
snap_CA                0
snap_TX                0
snap_WI                0
sell_price      12299413
dtype: int64

id              0.000000
item_id         0.000000
dept_id         0.000000
cat_id          0.000000
store_id        0.000000
state_id        0.000000
day             0.000000
demand          0.000000
part            0.000000
date            0.000000
wm_yr_wk        0.000000
weekday         0.000000
wday            0.000000
month           0.000000
year            0.000000
event_name_1    0.917725
event_type_

,id,item_id,dept_id,cat_id,store_id,state_id,day,demand,part,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,train,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,train,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,train,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,train,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,train,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN


As we stated above, it is expected to have this many null values in sell_price.
We were expecting 1,757,059 weeks without price informantion, so 1,757,059 x 7 = 12,299,413

In [14]:
#make new variables
# time related variables
''' This piece of code is ok. Just comented it to test other things.
data['date'] = pd.to_datetime(data['date'])
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['week'] = data['date'].dt.week
data['day'] = data['date'].dt.day
data['dayofweek'] = data['date'].dt.dayofweek

gc.collect()
data['season'] = data.apply(lambda row: 'spring' if (row['month'] >= 3 and row['month'] <= 5)
                            else('summer' if (row['month'] >= 6 and row['month'] <= 8)
                            else('fall' if (row['month'] >= 9 and row['month'] <= 11)
                            else('winter'))), axis = 1)

data.head()
'''

" This piece of code is ok. Just comented it to test other things.\ndata['date'] = pd.to_datetime(data['date'])\ndata['year'] = data['date'].dt.year\ndata['month'] = data['date'].dt.month\ndata['week'] = data['date'].dt.week\ndata['day'] = data['date'].dt.day\ndata['dayofweek'] = data['date'].dt.dayofweek\n\ngc.collect()\ndata['season'] = data.apply(lambda row: 'spring' if (row['month'] >= 3 and row['month'] <= 5)\n                            else('summer' if (row['month'] >= 6 and row['month'] <= 8)\n                            else('fall' if (row['month'] >= 9 and row['month'] <= 11)\n                            else('winter'))), axis = 1)\n\ndata.head()\n"

In [15]:
#Verify the correlation between products
def correlation_between_products(df, filter_1, filter_2, name, n = 10):
    """Print numer of products being analised;
    print the top n correlations;
    plots the heatmap;
    save the heatmap as name.png

    Usage
    ------

    correlation_between_products(df, filter_1, filter_2, n, name)
    filter_ and filter_2 are conditions, e.g. filter_1 = data['store_id'] == 'CA_4'
                                              filter_2 = data['dept_id'] == 'HOBBIES_2'
    name is the name of the file with the heatmap that will be created as a .png
    n is an int 
    """
    products_analised = len(df[(filter_1) & (filter_2)]['id'].drop_duplicates())
    print('Products:', products_analised)
    print()

    data_filtered = df[(filter_1) & (filter_2)][['date', 'id', 'demand']].copy()
    data_filtered = data_filtered.pivot(index='date', columns='id', values='demand')
    corr_matrix = data_filtered.corr()

    def get_redundant_pairs(df):
        '''Get diagonal and lower triangular pairs of correlation matrix'''
        pairs_to_drop = set()
        cols = df.columns
        for i in range(0, df.shape[1]):
            for j in range(0, i+1):
                pairs_to_drop.add((cols[i], cols[j]))
        return pairs_to_drop

    corr_table = corr_matrix.abs().unstack()
    labels_to_drop = get_redundant_pairs(data_filtered)
    corr_table = corr_table.drop(labels = labels_to_drop).sort_values(ascending = False)

    print("Top Absolute Correlations")
    print(corr_table[:n + 1])
    print()

    size_x = products_analised     #This is a good size to visualise the heatmap saved as .png
    size_y = products_analised
    plt.figure(figsize = (size_x, size_y))
    sns.set(font_scale = 1.5)

    ax = sns.heatmap(corr_matrix, annot = True, linewidth = 0.5, cmap='coolwarm')
    bottom, top = ax.get_ylim()
    ax.set_ylim(bottom + 0.5, top - 0.5)

    plt.tight_layout()
    plt.savefig(name+ '.png')
    
    #fig.set_visible(not fig.get_visible())  #This line is just to make this notebook lighter.
    #plt.draw()                              #This line is just to make this notebook lighter.
    #plt.show()                              #This line is commented just to make this notebook lighter.
    plt.close('all')                         #This line is just to make this notebook lighter.

    return

In [25]:
#correlation_between_products(data, data['store_id'] == 'CA_1', data['dept_id'] == 'HOBBIES_2', 'store_CA_1_department_HOBBIES_2', 10)

Products: 149

Top Absolute Correlations
id                             id                           
HOBBIES_2_071_CA_1_validation  HOBBIES_2_147_CA_1_validation    0.426263
HOBBIES_2_058_CA_1_validation  HOBBIES_2_126_CA_1_validation    0.407455
HOBBIES_2_087_CA_1_validation  HOBBIES_2_147_CA_1_validation    0.388825
                               HOBBIES_2_091_CA_1_validation    0.386128
HOBBIES_2_071_CA_1_validation  HOBBIES_2_091_CA_1_validation    0.382073
HOBBIES_2_091_CA_1_validation  HOBBIES_2_147_CA_1_validation    0.325895
HOBBIES_2_064_CA_1_validation  HOBBIES_2_147_CA_1_validation    0.318753
HOBBIES_2_034_CA_1_validation  HOBBIES_2_049_CA_1_validation    0.318606
HOBBIES_2_043_CA_1_validation  HOBBIES_2_109_CA_1_validation    0.310079
                               HOBBIES_2_095_CA_1_validation    0.301492
HOBBIES_2_071_CA_1_validation  HOBBIES_2_087_CA_1_validation    0.295804
HOBBIES_2_021_CA_1_validation  HOBBIES_2_099_CA_1_validation    0.291820
HOBBIES_2_064_CA_1_val

In [26]:
#correlation_between_products(data, data['store_id'] == 'CA_2', data['dept_id'] == 'HOBBIES_2', 'store_CA_2_department_HOBBIES_2', 10)

Products: 149

Top Absolute Correlations
id                             id                           
HOBBIES_2_071_CA_2_validation  HOBBIES_2_091_CA_2_validation    0.463365
HOBBIES_2_087_CA_2_validation  HOBBIES_2_091_CA_2_validation    0.446427
HOBBIES_2_064_CA_2_validation  HOBBIES_2_091_CA_2_validation    0.441888
HOBBIES_2_091_CA_2_validation  HOBBIES_2_147_CA_2_validation    0.381800
HOBBIES_2_043_CA_2_validation  HOBBIES_2_121_CA_2_validation    0.379960
HOBBIES_2_071_CA_2_validation  HOBBIES_2_147_CA_2_validation    0.358865
HOBBIES_2_034_CA_2_validation  HOBBIES_2_049_CA_2_validation    0.355294
HOBBIES_2_064_CA_2_validation  HOBBIES_2_071_CA_2_validation    0.335379
HOBBIES_2_024_CA_2_validation  HOBBIES_2_043_CA_2_validation    0.329893
HOBBIES_2_010_CA_2_validation  HOBBIES_2_091_CA_2_validation    0.324645
HOBBIES_2_087_CA_2_validation  HOBBIES_2_147_CA_2_validation    0.323868
HOBBIES_2_034_CA_2_validation  HOBBIES_2_055_CA_2_validation    0.322899
HOBBIES_2_014_CA_2_val

In [27]:
#correlation_between_products(data, data['store_id'] == 'CA_3', data['dept_id'] == 'HOBBIES_2', 'store_CA_3_department_HOBBIES_2', 10)

Products: 149

Top Absolute Correlations
id                             id                           
HOBBIES_2_087_CA_3_validation  HOBBIES_2_091_CA_3_validation    0.493718
HOBBIES_2_091_CA_3_validation  HOBBIES_2_147_CA_3_validation    0.482193
HOBBIES_2_071_CA_3_validation  HOBBIES_2_091_CA_3_validation    0.446407
HOBBIES_2_087_CA_3_validation  HOBBIES_2_147_CA_3_validation    0.442257
HOBBIES_2_038_CA_3_validation  HOBBIES_2_043_CA_3_validation    0.421248
HOBBIES_2_058_CA_3_validation  HOBBIES_2_126_CA_3_validation    0.405004
HOBBIES_2_024_CA_3_validation  HOBBIES_2_058_CA_3_validation    0.370106
HOBBIES_2_071_CA_3_validation  HOBBIES_2_087_CA_3_validation    0.358696
HOBBIES_2_126_CA_3_validation  HOBBIES_2_141_CA_3_validation    0.339128
HOBBIES_2_024_CA_3_validation  HOBBIES_2_141_CA_3_validation    0.338040
HOBBIES_2_058_CA_3_validation  HOBBIES_2_141_CA_3_validation    0.324488
HOBBIES_2_021_CA_3_validation  HOBBIES_2_147_CA_3_validation    0.322019
HOBBIES_2_071_CA_3_val

In [28]:
#correlation_between_products(data, data['store_id'] == 'CA_4', data['dept_id'] == 'HOBBIES_2', 'store_CA_4_department_HOBBIES_2', 10)

Products: 149

Top Absolute Correlations
id                             id                           
HOBBIES_2_091_CA_4_validation  HOBBIES_2_147_CA_4_validation    0.405509
HOBBIES_2_018_CA_4_validation  HOBBIES_2_141_CA_4_validation    0.332974
HOBBIES_2_087_CA_4_validation  HOBBIES_2_147_CA_4_validation    0.324804
                               HOBBIES_2_091_CA_4_validation    0.316192
HOBBIES_2_071_CA_4_validation  HOBBIES_2_091_CA_4_validation    0.308538
HOBBIES_2_064_CA_4_validation  HOBBIES_2_087_CA_4_validation    0.300614
HOBBIES_2_010_CA_4_validation  HOBBIES_2_091_CA_4_validation    0.286140
HOBBIES_2_101_CA_4_validation  HOBBIES_2_123_CA_4_validation    0.278259
HOBBIES_2_064_CA_4_validation  HOBBIES_2_091_CA_4_validation    0.276275
HOBBIES_2_021_CA_4_validation  HOBBIES_2_091_CA_4_validation    0.272604
HOBBIES_2_106_CA_4_validation  HOBBIES_2_147_CA_4_validation    0.265120
HOBBIES_2_049_CA_4_validation  HOBBIES_2_055_CA_4_validation    0.262096
HOBBIES_2_014_CA_4_val

The correct would be to do the correlation matrix for all department and stores.
Moreover, it would be interesting to do the corretlation matrix for the same product in diferent stores.
However, this analysis is mainly to see if we really need one model for product.

Considering that the highest correlation between products is 0.49. We decided to choose one model, but calibrate it individually per product.

We will try to model using ARIMA(p, d, q). Nonetheless, first, we will see if the series are stationary and set d = 1 for those that are not stationary. Second, if we did not achieve stationarity, we will see if we need to deseasonalize the series. After, we will calculate the autocorrelation for each series to calibrate the 'p' parameter. Last, we will calibrate the 'q' parameter.






How to calibrat q, set to 28 just because our model is for 28 days????



In [16]:
'''
This code takes too long to run in a personal computer. 
However, you can find the result of the ADF test in https://github.com/MaxWienandts/M5-Forecasting

#Augmented Dickey-Fuller test.
#If p-value is greater than 0.01, we will consider that the series is non-stationary and repeat the test using the first difference.
products_ids = data['id'].unique()
ADF_test = []
columns_ADF_test = ['product_id', 'ADF_test_diff_0', 'ADF_test_diff_1', 'diff_10%', 'diff_5%', 'diff_1%']
progress = 0   #Usefull to see the progress of the code
progress_1000 = 1
for e in products_ids:
    ADF_test_array = data[data['id'] == e]['demand']
    ADF_test_diff_0 = round(adfuller(ADF_test_array)[1], 4)
    ADF_test_diff_1 = round(adfuller(ADF_test_array.diff(periods = 1)[1:])[1], 4)
    diff_10 = 0    #Consider different p-values to accept or reject the null hypothesis.
    diff_05 = 0
    diff_01 = 0
    if ADF_test_diff_0 > 0.01:
        diff_01 = 1
        if ADF_test_diff_1 > 0.01: # See if just the first difference is enouth to obtain a stationary series.  
            diff_01 = 2
    if ADF_test_diff_0 > 0.05:
        diff_05 = 1
        if ADF_test_diff_1 > 0.05: # See if just the first difference is enouth to obtain a stationary series.  
            diff_05 = 2
    if ADF_test_diff_0 > 0.1:
        diff_10 = 1
        if ADF_test_diff_1 > 0.1: # See if just the first difference is enouth to obtain a stationary series.  
            diff_10 = 2
    ADF_test.append([e, ADF_test_diff_0, ADF_test_diff_1, diff_10, diff_05, diff_01])
    
    progress += 1
    if progress > progress_1000 * 1000:
        df_ADF_test_partial = pd.DataFrame(ADF_test, columns = columns_ADF_test)
        df_ADF_test_partial.to_csv('df_ADF_test_partial.csv')
        print(progress / len(products_ids))
        progress_1000 +=1
    

df_ADF_test = pd.DataFrame(ADF_test, columns = columns_ADF_test)
df_ADF_test.to_csv('ADF_test.csv')
df_ADF_test.head()


'''

0.03283043620859298
0.06562807477861594
0.0984257133486389
0.13122335191866186
0.16402099048868482
0.19681862905870778
0.22961626762873072
0.2624139061987537
0.29521154476877665
0.3280091833387996
0.3608068219088226
0.3936044604788455
0.4264020990488685
0.45919973761889143
0.49199737618891437
0.5247950147589373
0.5575926533289604
0.5903902918989833
0.6231879304690062
0.6559855690390292
0.6887832076090521
0.7215808461790751
0.7543784847490981
0.787176123319121
0.819973761889144
0.852771400459167
0.8855690390291899
0.9183666775992129
0.9511643161692358
0.9839619547392587


,product_id,ADF_test_diff_0,ADF_test_diff_1,diff_10%,diff_5%,diff_1%
0,HOBBIES_1_001_CA_1_validation,0.0502,0.0,0,1,1
1,HOBBIES_1_002_CA_1_validation,0.0000,0.0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0.0004,0.0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0.0004,0.0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0.0000,0.0,0,0,0


In [ ]:
df_ADF_test = pd.read_csv("ADF_test.csv")

In [19]:
#Verify ADF test

#seasonality and trend
seasonality = df_ADF_test[df_ADF_test['diff_1%'] == 2][['diff_10%', 'diff_5%', 'diff_1%']].sum()
seasonality

diff_10%    0
diff_5%     0
diff_1%     0
dtype: int64

As the Adf test shows, we do not need to worry about seasonality or trend.
Just take to first difference is enough to turn our data stationary. 

In [ ]:
#Verify the ADF test by significance level.